In [ ]:
# DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
# SERVER_NAME = 'DESKTOP-4AH43BG'
# DATABASE_NAME = 'PipelineDB'
# username = 'sa'
# password = 'Sagar@2570'

# connection_string=f"""
# DRIVER={{{DRIVER_NAME}}};
# SERVER={SERVER_NAME};
# DATABASE={DATABASE_NAME};
# Trust_Connection=yes;
# UID={username};
# PWD={password};
# """
# conn=odbc.connect(connection_string)
# print(conn)

In [ ]:
#import required libraries
import pandas as pd
from sqlalchemy import create_engine
import urllib

In [ ]:
#connecting to sql server 
username = 'sa'
password = 'Sagar@2570'
server = 'DESKTOP-4AH43BG'
database = 'PipelineDB'
driver = 'ODBC Driver 17 for SQL Server' 

# using sqlalchemy for connecting
params = urllib.parse.quote_plus(
    f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Load CSVs
customer = pd.read_csv("cleaned_customer_csv.csv")
transaction = pd.read_csv("cleaned_transaction_csv.csv")
orders = pd.read_csv("order_data.csv")





In [ ]:
# Upload to sql database
customer.to_sql('Customers', con=engine, if_exists='replace', index=False)
transaction.to_sql('Transactions', con=engine, if_exists='replace', index=False)
orders.to_sql('Orders', con=engine, if_exists='replace', index=False)

In [ ]:
#fetching data from sql
read_customer_sql = pd.read_sql_table('Customers',con=engine)
read_orders_sql = pd.read_sql_table('Orders',con = engine)
read_transactions_sql = pd.read_sql_table('Transactions',con = engine)


In [ ]:
#merging tables
merge_cust_order = pd.merge(read_customer_sql,read_orders_sql,on='customer_id')

In [ ]:
#split full name into firstname and lastname 
merge_cust_order[['firstName', 'lastName']] = merge_cust_order['name'].str.split(' ', n=1, expand=True)
merge_cust_order['lastName'] = merge_cust_order['lastName'].str.split(' ').str[0]


In [ ]:
#get unique status
merge_cust_order['loyalty_status'].unique()

In [ ]:
#mapping loyalty_status
customer_mapping = {
    'Gold' : 2,
    'Silver' : 1,
    'Bronze' : 0
}

merge_cust_order['Customer_Tier'] = merge_cust_order['loyalty_status'].map(customer_mapping)

In [ ]:
#relocating columns

cols = merge_cust_order.columns.tolist()
cols.remove('firstName')
cols.insert(2,'firstName')
merge_cust_order = merge_cust_order[cols]

cols = merge_cust_order.columns.tolist()
cols.remove('lastName')
cols.insert(3,'lastName')
merge_cust_order = merge_cust_order[cols]



In [ ]:
#pushing filtered data to sql server
merge_cust_order.to_sql('unifiedCustomerOrders',con = engine,if_exists='replace',index=False)